# Inference Time Study

In [ ]:
################################################## Initialize ##################################################

# Add the new path
import sys
new_path2 = "/home/michele/code/michele_mmdet3d/"
if not new_path2 in sys.path:
    sys.path.insert(1, new_path2)

# TODO:
#   - Try to create the loader/packer/preprocessor with the config file, and not manually inserting the parameters
#   - Need to implement all the strange categories that Riccardo did?

# Main variables
    # Boolean to print inputs/outputs of each stage
wanna_print_in_out = False
    # Home directory within ADE
home_dir = '/home/michele/code/'
    # Relative path from "home_dir" to the .bin files
path_from_home_to_bin_files = "michele_mmdet3d/data/minerva_polimove/training/velodyne"
    # Path to the ".txt" file in "ImageSets", containing the list of validation files
val_list_txt_file = "/home/michele/code/michele_mmdet3d/data/minerva_polimove/ImageSets/val.txt"



############################################# Time-related variables #############################################
import time
import torch

delta_preprocessing = []
delta_inference = []
delta_postprocessing = []



################################################ Create the model ################################################
from mmdeploy.apis.inference import get_model

# Build the model with the built-in function
model = get_model(
    model_cfg= home_dir + 'michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py',
    deploy_cfg= home_dir + 'mmdeploy/configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4.py',
    backend_files= [ home_dir + 'mmdeploy/mmdeploy_models/minerva_pointpillars/end2end.engine' ],
    img= home_dir + 'michele_mmdet3d/data/minerva_polimove/training/velodyne/1723235584511788288.bin',
    device='cuda'
)

# Build the additional BBoxHead (for PointPillars)
from mmengine.registry import MODELS
bbox_head = MODELS.build(model.model_cfg.model['bbox_head'])



############################################### Automatic configs ###############################################

config_dictionary = {}

# For the loader
config_dictionary['LoadPointsFromFile'] = {
    'coord_type': model.model_cfg.val_dataloader.dataset.pipeline[0].coord_type,
    'load_dim': model.model_cfg.val_dataloader.dataset.pipeline[0].load_dim,
    'use_dim': model.model_cfg.val_dataloader.dataset.pipeline[0].use_dim
}

# For the packer
#  |
#  |--------------> NOTE: Not used because the only field is 
#  |                "keys" and we have it DIFFERENT
#  |
config_dictionary['Pack3DDetInputs'] = {}

# For the Det3DDataPreprocessor
config_dictionary['Det3DDataPreprocessor'] = {
    'voxel': model.model_cfg.model.data_preprocessor.voxel,
    'voxel_layer': dict(
        max_num_points = model.model_cfg.model.data_preprocessor.voxel_layer.max_num_points,
        max_voxels = model.model_cfg.model.data_preprocessor.voxel_layer.max_voxels,
        point_cloud_range = model.model_cfg.model.data_preprocessor.voxel_layer.point_cloud_range,
        voxel_size = model.model_cfg.model.data_preprocessor.voxel_layer.voxel_size
    )
}



############################################ Basic type of input metas ############################################

# NOTE:
#   - Necessary to use the bbox_head with no errors
#   - Supposes that no transformations are applied to the pointcloud

import torch
import numpy as np
from mmdet3d.structures.bbox_3d import Box3DMode, LiDARInstance3DBoxes

# Recreating the variable with the correct types
metas_base_variable = {
    'box_mode_3d': Box3DMode.LIDAR,  # Enum type
    'lidar_path': '',
    'pcd_vertical_flip': False,
    'pcd_rotation_angle': 0.0,
    'pcd_rotation': torch.tensor([[1., 0., 0.],
                                  [-0., 1., 0.],
                                  [0., 0., 1.]]),
    'transformation_3d_flow': ['R', 'S', 'T'],
    'pcd_scale_factor': 1.0,
    'box_type_3d': LiDARInstance3DBoxes,
    'pcd_trans': np.array([0., 0., 0.]),
    'pcd_horizontal_flip': False,
    'flip': False,
    'axis_align_matrix': np.array([[1., 0., 0., 0.],
                                   [0., 1., 0., 0.],
                                   [0., 0., 1., 0.],
                                   [0., 0., 0., 1.]])
}



############################################ Define testing configuration ############################################

# NOTE:
#   - Necessary to use the bbox_head with no errors
#   - Can be modified as wanted

from mmengine.config.config import ConfigDict

test_cfg = ConfigDict(
    max_num = 10, 
    min_bbox_size = 0, 
    nms_across_levels = False, 
    nms_pre = 10, 
    nms_thr = 0.01, 
    score_thr = 0.3, 
    use_rotate_nms = True
)

In [2]:
############################################### Very original input ###############################################

# Read the names of the validation files from the ".txt" file in "ImageSets"
with open(val_list_txt_file, 'r') as file:
    val_file_names = [line.strip()+".bin" for line in file]

# Create the inputs
#   - List of strings
#   - Each string is the ABSOLUTE path to the ".bin" file
inputs = []
for file_name in val_file_names:
    inputs.append(home_dir+path_from_home_to_bin_files+"/"+file_name)

In [3]:
############################################### LoadPointsFromFile ###############################################

from mmdet3d.datasets.transforms.loading import LoadPointsFromFile

# Initialize the loader
loader = LoadPointsFromFile(
    coord_type=config_dictionary['LoadPointsFromFile']['coord_type'],
    load_dim=config_dictionary['LoadPointsFromFile']['load_dim'],
    use_dim=config_dictionary['LoadPointsFromFile']['use_dim']
)

# Print the input to this stage
if wanna_print_in_out:
    print("\nLoadPointsFromFile input:")
    for element in inputs:
        print(element)

# For cycle to also handle lists of inputs
for i in range(len(inputs)):
    
    # Prepare the string input for the loader (needs two dictionaries, one nested into the other)
    inputs[i] = dict(
        lidar_points=dict(
            lidar_path=inputs[i]
        )
    )

    # Actual modification of the dictionary
    loader(inputs[i])

# Print the output of this stage
if wanna_print_in_out: 
    print("\nLoadPointsFromFile output:")
    for element in inputs:
        print(element)

In [4]:
############################################### Pack3DDetInputs ###############################################

from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

# Initialize the packer
packer = Pack3DDetInputs(keys='points')

# Print the input to this stage
if wanna_print_in_out: 
    print("\nPack3DDetInputs input:")
    for element in inputs:
        print(element)

# For cycle to also handle lists of inputs
for i in range(len(inputs)):
    
    # Actual modification of the dictionary
    packer(inputs)

# Print the output of this stage
if wanna_print_in_out: 
    print("\nPack3DDetInputs output:")
    for element in inputs:
        print(element)

In [5]:
############################################### Det3DDataPreprocessor ###############################################

from mmdet3d.models.data_preprocessors.data_preprocessor import Det3DDataPreprocessor

# Initialize the preprocessor
preprocessor = Det3DDataPreprocessor(
    voxel=config_dictionary['Det3DDataPreprocessor']['voxel'],
    voxel_layer=config_dictionary['Det3DDataPreprocessor']['voxel_layer']
)

# Print the input to this stage
if wanna_print_in_out: 
    print("\nDet3DDataPreprocessor input:")
    for element in inputs:
        print(element)

# Create the list with the final inputs
final_inputs = []
for i in range(len(inputs)):    
    # Create a temporary dictionary to be passed to the preprocessor (in the right format)
    temp = dict(
        inputs=dict(
            points=[inputs[i]['points']]    ## Must be into a list, otherwise error
        )
    )
    # Take out the result of the Det3DDataPreprocessor, and also compute the time
    start_preprocessing = time.time()
    final_inputs.append(
        preprocessor(temp)
    )
    torch.cuda.synchronize()
    end_preprocessing = time.time()
    # Append the time to the right list
    delta_preprocessing.append(end_preprocessing-start_preprocessing)

# Print the output of this stage
if wanna_print_in_out: 
    print("\nDet3DDataPreprocessor output:")
    for element in final_inputs:
        print(element)

In [6]:
################################################# Inference #################################################

raw_results=[]
for element in final_inputs:
    # Take out the result of the inferencer, and also compute the time
    start_inference = time.time()
    raw_results.append(
        model(element['inputs'], mode='tensor')
    )
    torch.cuda.synchronize()
    end_inference = time.time()
    # Append the time to the right list
    delta_inference.append(end_inference-start_inference)

In [ ]:
############################################## Post_processing ##############################################

predictions=[]
for element in raw_results:
    # Take out the prediction, and also compute the time
    start_postprocessing = time.time()
    predictions.append(
        bbox_head.predict_by_feat(
            cls_scores = element['cls_score'],
            bbox_preds = element['bbox_pred'],
            dir_cls_preds = element['dir_cls_pred'],
            batch_input_metas = [metas_base_variable],
            cfg = test_cfg
        )
    )
    torch.cuda.synchronize()
    end_postprocessing = time.time()
    # Append the time to the right list
    delta_postprocessing.append(end_postprocessing-start_postprocessing)

In [ ]:
#################################################################################################################
#                                            PLOTTING OF FREQUENCY                                              #
#################################################################################################################

import matplotlib.pyplot as plt

def print_frequency_plot(values, title = "Frequency Plot"):
    # Calculating the average value of the given list
    average_value = sum(values) / len(values)

    # Plotting the frequency plot with a vertical red line for the average value
    plt.figure(figsize=(10, 6))
    plt.hist(values, bins=20, alpha=0.7, color='blue', edgecolor='black')
    plt.axvline(average_value, color='red', linestyle='dashed', linewidth=2, label=f'Average: {average_value:.4f}')
    plt.title(title)
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)
    plt.show()

# delta_inference = []
# for i in range(len(delta_voxel)):
#     delta_inference.append(delta_voxel[i] + delta_middle[i] + delta_backbone[i] + delta_neck[i])
print_frequency_plot(delta_post, "Postprocessing time")